In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('online_shoppers_intention.csv')
data.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


In [3]:
buy = (data[data['Revenue']==True])
no_buy = (data[data['Revenue']==False]).sample(n=2000,random_state=1)
data = pd.concat([buy,no_buy], axis=0)
categorical_variable = ['Month', 'VisitorType']
data = pd.get_dummies(data=data, columns = categorical_variable)

In [4]:
features = [x for x in data.columns if x not in ['Revenue','BounceRates','ExitRates','PageValues','OperatingSystems','Browser','Region','TrafficType']]

In [5]:
X = data[features]
y = data['Revenue']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=0)
print('X Train shape {}'.format(X_train.shape))
print('X Test shape {}'.format(X_test.shape))

X Train shape (3126, 21)
X Test shape (782, 21)


In [7]:
from sklearn.preprocessing import StandardScaler
Z_score_scaler = StandardScaler().fit(X_train)


#Transform the training and test data using the scaler
X_train_scaled = Z_score_scaler.transform(X_train)
X_test_scaled = Z_score_scaler.transform(X_test)


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.activations import relu,linear,sigmoid
from tensorflow.keras.losses import BinaryCrossentropy,MeanSquaredError
from tensorflow.keras.optimizers import Adam

In [9]:
tf.random.set_seed(1234)  # applied to achieve consistent results

model = Sequential(
    [
     tf.keras.Input(shape=(21,)),               
     Dense(units = 100, activation = "sigmoid"),
     Dense(units = 40, activation = "sigmoid"),
     Dense(units = 1, activation = "sigmoid")
    ] 
)   


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               2200      
                                                                 
 dense_1 (Dense)             (None, 40)                4040      
                                                                 
 dense_2 (Dense)             (None, 1)                 41        
                                                                 
Total params: 6281 (24.54 KB)
Trainable params: 6281 (24.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)


In [12]:
model.fit(
    X_train_scaled, y_train,
    epochs=100
)


Epoch 1/100
98/98 [==============================] - 1s 2ms/step - loss: 0.6753
Epoch 2/100
98/98 [==============================] - 0s 868us/step - loss: 0.6310
Epoch 3/100
98/98 [==============================] - 0s 856us/step - loss: 0.6221
Epoch 4/100
98/98 [==============================] - 0s 804us/step - loss: 0.6209
Epoch 5/100
98/98 [==============================] - 0s 794us/step - loss: 0.6217
Epoch 6/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6180
Epoch 7/100
98/98 [==============================] - 0s 814us/step - loss: 0.6178
Epoch 8/100
98/98 [==============================] - 0s 845us/step - loss: 0.6153
Epoch 9/100
98/98 [==============================] - 0s 905us/step - loss: 0.6148
Epoch 10/100
98/98 [==============================] - 0s 938us/step - loss: 0.6131
Epoch 11/100
98/98 [==============================] - 0s 880us/step - loss: 0.6134
Epoch 12/100
98/98 [==============================] - 0s 897us/step - loss: 0.6133
Epoch 13/100
98/9

In [13]:
from sklearn.metrics import f1_score
y_train_predict = model.predict(X_train_scaled)
y_train_predict = (y_train_predict >=0.5).astype(int)
train_accuracy = f1_score(y_train,y_train_predict)

y_test_predict = model.predict(X_test_scaled)
y_test_predict = (y_test_predict >=0.5).astype(int)

test_accuracy = f1_score(y_test,y_test_predict)
print('train f1 score {}'.format(train_accuracy))
print('test f1 score {}'.format(test_accuracy))

25/25 [==============================] - 0s 1000us/step
train f1 score 0.7060942659861904
test f1 score 0.6856465005931198
